## Load, clean, and combine TCGA-DLBC RNA-seq count files

This notebook processes the raw RNA-seq gene expression count files downloaded from the Genomic Data Commons (GDC) for the TCGA-DLBC project. The goal of this step is to convert the individual STAR-generated count files into a single, tidy gene-by-sample count matrix suitable for downstream normalization and analysis.

The notebook reads all STAR “Counts” TSV files from the local data directory, removes non-gene summary rows produced by the STAR workflow, and cleans gene identifiers by removing Ensembl version suffixes. For each file, unstranded gene counts are extracted and stored as a pandas Series indexed by gene ID. These series are then concatenated column-wise to form a matrix with genes as rows and tumor samples as columns.

Sample-level quality control is performed by computing library sizes, defined as the total number of sequencing reads per sample. This provides a basic check on sequencing depth and ensures that all samples have adequate coverage. Column names are then cleaned and standardized by mapping file identifiers to filenames using the GDC manifest and extracting TCGA sample barcodes to produce interpretable and consistent sample labels.

Finally, the cleaned gene-by-sample count matrix is saved in an efficient columnar format (Parquet) for use in subsequent normalization, exploratory analysis, predictive modeling, and inferential procedures. This step establishes a reproducible and tidy representation of the raw gene expression data.

In [4]:
from pathlib import Path
import pandas as pd
import re

tsv_paths = sorted(Path("../data/gdc").rglob("*.tsv"))
len(tsv_paths)

48

Define loader function

In [11]:
def read_counts_tsv(path: Path) -> pd.Series:
    # Skip metadata lines that start with '#'
    df = pd.read_csv(path, sep="\t", comment="#")

    # Drop STAR-style summary rows (they are N_* not __*)
    df = df[~df["gene_id"].astype(str).str.startswith("N_")].copy()

    # Clean Ensembl version suffix
    genes = df["gene_id"].astype(str).str.replace(r"\.\d+$", "", regex=True)

    # Use unstranded gene counts
    counts = pd.to_numeric(df["unstranded"], errors="raise")

    return pd.Series(counts.values, index=genes.values, name=path.parent.name)


Read and combine all samples

In [18]:
series_list = [read_counts_tsv(p) for p in tsv_paths]
counts = pd.concat(series_list, axis=1)

counts.shape

(60660, 48)

In [19]:
library_sizes = counts.sum(axis=0)
library_sizes.describe()

count    4.800000e+01
mean     4.348995e+07
std      1.058509e+07
min      2.327589e+07
25%      3.832623e+07
50%      4.255281e+07
75%      5.206553e+07
max      6.433010e+07
dtype: float64

In [20]:
manifest = pd.read_csv(
    "../manifests/tcga_dlbc_star_counts_manifest.tsv",
    sep="\t"
)

id_to_filename = dict(zip(manifest["id"], manifest["filename"]))

counts = counts.rename(columns=id_to_filename)
counts.columns[:5]

Index(['6f4e8f95-44f7-41c5-8af9-19212eff36d6.rna_seq.augmented_star_gene_counts.tsv',
       '2fb12ae8-cb05-41bb-a567-0c7e300bb07e.rna_seq.augmented_star_gene_counts.tsv',
       'f7a04906-7cb5-4f46-a885-699969be5405.rna_seq.augmented_star_gene_counts.tsv',
       '5ae63973-9111-43c9-9220-7fd7510294e1.rna_seq.augmented_star_gene_counts.tsv',
       'c70162ed-dafc-4a4c-b7dc-354d2c06b2c9.rna_seq.augmented_star_gene_counts.tsv'],
      dtype='object')

Extract TCGA barcodes

In [21]:
def extract_tcga_barcode(s: str) -> str:
    m = re.search(r"(TCGA-[A-Z0-9]{2}-[A-Z0-9]{4})", s)
    return m.group(1) if m else s

counts = counts.rename(
    columns={c: extract_tcga_barcode(c) for c in counts.columns}
)

counts.columns[:10]

Index(['6f4e8f95-44f7-41c5-8af9-19212eff36d6.rna_seq.augmented_star_gene_counts.tsv',
       '2fb12ae8-cb05-41bb-a567-0c7e300bb07e.rna_seq.augmented_star_gene_counts.tsv',
       'f7a04906-7cb5-4f46-a885-699969be5405.rna_seq.augmented_star_gene_counts.tsv',
       '5ae63973-9111-43c9-9220-7fd7510294e1.rna_seq.augmented_star_gene_counts.tsv',
       'c70162ed-dafc-4a4c-b7dc-354d2c06b2c9.rna_seq.augmented_star_gene_counts.tsv',
       '528116e8-c133-49c9-a532-18ad99a6fe52.rna_seq.augmented_star_gene_counts.tsv',
       '70510d37-dc1a-433b-84ee-d0f7d9ca1c3d.rna_seq.augmented_star_gene_counts.tsv',
       'cba52f4f-eb3a-4475-8735-d5c9fd355b3d.rna_seq.augmented_star_gene_counts.tsv',
       'e492067e-bded-4222-a2be-302f6d39160e.rna_seq.augmented_star_gene_counts.tsv',
       '18ba7bf2-59d3-4c59-acdc-088cf3d4e5a6.rna_seq.augmented_star_gene_counts.tsv'],
      dtype='object')

Save the matrix

In [22]:
counts.to_parquet("../data/dlbc_star_counts.parquet")